In [4]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%autoreload 2

import os, sys
from os.path import join, abspath
import time
import pdb
import glob

import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.misc as sc
from multiprocessing import Pool
import cv2

import ctypes as ct


ImportError: No module named 'shapenet_taxonomy'

In [7]:
sys.path.append('../utils')
sys.path.append('../chamfer_utils')

In [10]:
from helper_funcs import create_folder, remove_outliers
from shapenet_taxonomy import shapenet_category_to_id